# calculate rotation and transposition matrix

In [5]:
data_folder = r'D:\Shiwei\20210503-P_Forebrain_CTP09_from0423smRNA_cleared'
before_position_file = os.path.join(data_folder, '10x_positions_before.txt')
after_position_file = os.path.join(data_folder, '10x_positions_after.txt')


In [6]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [7]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


In [8]:
R, T

(array([[ 0.99997233,  0.00743838],
        [-0.00743838,  0.99997233]]), array([ 127.58025184, 1222.50281245]))

# transpose 60x positions

In [9]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [11]:
old_positions

array([[-2540.9,   201.8],
       [-2540.9,   423. ],
       [-2540.9,   644.2],
       [-2540.9,   865.3],
       [-2540.9,  1086.5],
       [-2540.9,  1307.7],
       [-2319.7,  1307.7],
       [-2319.7,  1086.5],
       [-2319.7,   865.3],
       [-2319.7,   644.2],
       [-2319.7,   423. ],
       [-2319.7,   201.8],
       [-2098.5,   201.8],
       [-2098.5,   423. ],
       [-2098.5,   644.2],
       [-2098.5,   865.3],
       [-2098.5,  1086.5],
       [-2098.5,  1307.7],
       [-1877.4,  1307.7],
       [-1877.4,  1086.5],
       [-1877.4,   865.3],
       [-1877.4,   644.2],
       [-1877.4,   423. ],
       [-1877.4,   201.8],
       [-1656.2,   201.8],
       [-1656.2,   423. ],
       [-1656.2,   644.2],
       [-1656.2,   865.3],
       [-1656.2,  1086.5],
       [-1656.2,  1307.7],
       [-1435. ,  1307.7],
       [-1435. ,  1086.5],
       [-1435. ,   865.3],
       [-1435. ,   644.2],
       [-1435. ,   423. ],
       [-1435. ,   201.8],
       [-3259.9,  1655.7],
 

In [10]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-2414.75051952  1405.39704156]
 [-2416.3958899   1626.59092202]
 [-2418.04126028  1847.78480249]
 [-2419.68588682  2068.87868573]
 [-2421.3312572   2290.07256619]
 [-2422.97662758  2511.26644666]
 [-2201.78274711  2512.91181704]
 [-2200.13737673  2291.71793657]
 [-2198.49200635  2070.52405611]
 [-2196.84737981  1849.43017287]
 [-2195.20200943  1628.2362924 ]
 [-2193.55663905  1407.04241194]
 [-1972.36275858  1408.68778232]
 [-1974.00812896  1629.88166278]
 [-1975.65349934  1851.07554325]
 [-1977.29812588  2072.16942649]
 [-1978.94349626  2293.36330695]
 [-1980.58886664  2514.55718742]
 [-1759.49498341  2516.20181396]
 [-1757.84961303  2295.0079335 ]
 [-1756.20424265  2073.81405303]
 [-1754.55961611  1852.72016979]
 [-1752.91424573  1631.52628933]
 [-1751.26887535  1410.33240886]
 [-1530.07499488  1411.97777924]
 [-1531.72036526  1633.17165971]
 [-1533.36573564  1854.36554017]
 [-1535.01036218  2075.45942341]
 [-1536.65573256  2296.65330388]
 [-1538.30110294  2517.84718434]
 [-1317.10

In [12]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20210503-P_Forebrain_CTP09_from0423smRNA_cleared\translated_positions_all.txt


# further adjust manually

In [13]:
adjust_before = np.array([[-2414.75, 1405.4],[-1317.11, 2519.49],[-3147.84, 3296.2]])

adjust_after = np.array([[-2497.4, 1390.8],[-1398.6, 2504.3],[-3228.95, 3281.35]])


delta_adjust = adjust_after - adjust_before
delta_adjust

array([[-82.65, -14.6 ],
       [-81.49, -15.19],
       [-81.11, -14.85]])

In [14]:
manual_shift= np.mean(delta_adjust,axis=0)
manual_shift

array([-81.75, -14.88])

In [15]:
#manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [16]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20210503-P_Forebrain_CTP09_from0423smRNA_cleared\adjusted_translated_positions_all.txt
